# Functions and Structs

## Topics
- Interaction between functions and types
  - Duck-typing
  - Multiple dispatch
- Defining new types (structs)
- Mutating vs. non-mutating functions
- Unmutable and mutable structs

## Functions in a functional language
Functions are the building blocks of Julia code, acting as the subroutines, procedures, blocks, and similar structural concepts found in other programming languages.

- A function's job is to take a tuple of values as an argument list and return a value. 
- If the arguments contain mutable values like arrays, the array can be modified inside the function. 
    - ***By convention, an exclamation mark (!) at the end of a function's name indicates that the function may modify its arguments.***

## How to declare a function

Julia gives us a few different ways to write a function. The first (and most standard) requires `function` and `end` keywords.

In [ ]:
function sayhi(name)
    println("Hi $name, nice to meet you!")
end

sayhi("R2D2")

In [ ]:
function f(x)
    return x^2
end

f(42)

## Exercise 1: Addone

Write a function named `addone` that adds 1 to its input.

In [ ]:
function addone!(x)
    return x+=1
end

println("$(addone!(1))")
@show addone!(1);   # ; to hide the duplicate return value of the function

## Exercise 2: Polynomial

Write your own 2nd order polynomial function `poly2(x)` that evaluates $4 + 3x + 2x^2$.

In [ ]:
poly2(x) = 4+3x+2x^2

@show poly2(1);

Expand the function to take the polynomial coefficients as a parameter.

The function should take two parameters, `x` and `coeffs`. `coeffs` should be an array of length `3` that holds the coefficients of the polynomial. Internally your function should then compute $C_1 + C_2 x + C_3 x^2$ where $C_i$ is the $i$-th element of the `coeff` array.

In [ ]:
# see the solution that uses @assert to valid the length of coefficient arrays
function poly2(x, coeffs::Array)
    sum = 0
    for i in 1:length(coeffs)
        sum += coeffs[i]*x^i
    end
    return sum
    # return coeffs[1] + coeffs[2]x + coeffs[3]x^2
end

@show poly2(1, [4,3,2]);

### Single line function definitions
Alternatively, we could have spared a few lines of code and written:

In [ ]:
sayhi2(name) = println("Hi $name, nice to meet you!")

In [ ]:
f2(x) = x^2
@show f2(3)

### Anonymous functions

Anonymous functions are nice when, for example, passing a function as a parameter:

In [ ]:
x -> x^2

In [ ]:
function functional_square(f, x)
    f(f(x))
end

functional_square(x -> x^2, 2)

## Duck-typing in Julia
*"If it walks like a duck and quacks like a duck, it's a duck."*

Julia functions will just work on whatever input makes sense. For example, `sayhi` works also with the name written as an integer:

In [ ]:
sayhi(55595472)

And `f` will work on a matrix

In [ ]:
A = rand(3,3)

In [ ]:
f(A)

You can restrict the types of the function parameters usign `::Type`

In [ ]:
function g(x::Number)
    return x^4
end

A = rand(3,3)

# g(A) # This would not work
g(2)   # but this does

## Mutating vs. non-mutating functions
By convention, functions followed by `!` alter their contents and functions lacking `!` do not. This however, is just convention and changing the name of a function by adding `!` does not alter the function.

For example, let's look at the difference between `sort` and `sort!`.

In [ ]:
v = [3,5,2]

In [ ]:
sort(v)

In [ ]:
v

`sort(v)` returns a sorted array that contains the same elements as `v`, but `v` is left unchanged.

On the other hand, when we run `sort!(v)` the content of `v` is really modified.

In [ ]:
sort!(v)

In [ ]:
v

## Exercise 3: push?

Earlier we used a function called push!. Is there a function called push (without exclamation mark)?

In [ ]:
push!(v, 4)
vcat([1,2],[3])

## Structs

Structs are the way you declare **custom data types** in Julia. Essentially, a struct wraps one or more other datatypes inside and gives it a new name.

This is already a useful concept, but it gets much more powerful when we talk about multiple dispatch in functions.

You define a struct like this:

In [ ]:
struct MyInt
    number::Integer
    digit::Integer
end

So MyInt contains a single integer. You construct a new MyInt type variable by calling MyInt:

In [ ]:
myint = MyInt(5,1)

You can also now access the content of myint directly:

In [ ]:
myint.number
myint.digit

### Mutable

Notice that we cannot asign directly to `myint.number`:

In [ ]:
myint.number = 3

Structs are immutable by default. To change this, you can add the keyword `mutable`.

In [ ]:
mutable struct MyMutableInt
    number::Integer
end

mymutableint = MyMutableInt(4)
println(mymutableint)

mymutableint.number = 3
println(mymutableint)

### @enum

Here is a quick way of creating a new type with a small number of possible values.

In [6]:
@enum Fruit orange banana apple pineaple lemon

The new type is called `Fruit`. An object of type Fruit can only be one of the listed things.
Enum is short for enumerate. This is because, under the hood, each name is represented by a number (starts from 0).

In [7]:
myfruit = apple
myfruit

apple::Fruit = 2

In [8]:
typeof(myfruit)

Enum Fruit:
orange = 0
banana = 1
apple = 2
pineaple = 3
lemon = 4

## Epidemic Simulation

In the epidemic simulation, the board consists of individual plants in one of several states. 

On each time step, every plant will interact with each of it's neighbours. If the neighbour is infected, there is a chance this one will become infected as well.


First, let's create a datatype for a single plant. This needs to be mutable, since the plant can become infected and recover. We'll use `@enum` to list possible states and add a second variable, `infection_time`, to measure how long the plant has been infected.

In [ ]:
"Enumerate possible states of a single plant"
@enum InfectionStatus uninfected infected dead recovered

In [ ]:
"Data structure containing the infection status of a plant"
mutable struct Plant
    status::InfectionStatus
    infection_time::Int8
end

In [ ]:
# We can create a plant in any status
plant1 = Plant(uninfected, 0)
plant2 = Plant(infected, 0)

In [ ]:
# And we can change the infection status
plant1.status = infected

### Exercise 4: Time step

Write a function that takes three parameters:
  1. a plant
  2. recovery time (integer)
  2. the death rate of an infected plant (float)
  
The function updates the plant one time step. In one step, if the plant is infected
  1. it dies with the propability of the death rate
  2. the infection time increases by 1.
  3. if the infection time is greater than the maximum infection time, the plant recovers
  
Here we use a common trick to simulate a probability: get a random number between 0 and 1, and if it is smaller than the probability, run the plant dies.

In [ ]:
# define a function with the parameters plant, recovery_time and death_rate
function update!(plant::Plant, recovery_time, death_rate)
    
    # Do something only if the plant is infected
    if plant.status == infected
        
        # The plant dies with probability death_rate
        if rand() < death_rate
            # Kill the plant
            plant.status = dead
        end
        
        # add 1 to the infection time
        plant.infection_time += 1
        
        # if the infection time is larger than the recovery time, the plant recovers
        if plant.infection_time > recovery_time
            plant.status = recovered
        end
    end
end

In [ ]:
update!(plant2, 5, 0.02)
plant2

### Interaction

Next we need to implement the interaction between plants. Since we need this code later, we will wrap it in a function. If the other plant is infected, there is a chance this one becomes infected as well.

In [ ]:
"""Simulate an interaction between two plants. In the other plant is
   infected, it may infect this plant.
"""
function interact!(this_plant::Plant, other_plant::Plant, infection_rate)
    if this_plant.status == uninfected && other_plant.status == infected
        if rand() < infection_rate
            this_plant.status = infected
            this_plant.infection_time = 0
        end
    end
end

In [ ]:
plant1 = Plant(uninfected, 0)
plant2 = Plant(infected, 0)

interact!(plant1, plant2, 0.5)
println(plant1)

## Multiple dispatch
Until now, we have, in our examples, defined only functions with a single method having unconstrained argument types. 

Such functions behave just like they would in traditional dynamically typed languages. Nevertheless, we have used multiple dispatch almost continually without being aware of it: whenever we called a function with a different set of parameter types, Julia used a different version compiled for those parameters.

This is known as multiple dispatch!

### Methods

The word method has a slightly different meaning in Julia than in many other languages. In Julia, functions have methods.

We can define new functions with the same name as existing ones, and as long as they have different parameter types, Julia will know which one to call. The different versions of the function are called methods.

Still, this is very similar to the way many languages classes and objects and you to do everything class methods let you do. For example:

In [ ]:
function myfunc(x::Float64, y::Float64)
    return 2x + y
end

In [ ]:
# You can use the methods function to list the methods that currently exist for a function
methods(myfunc)

In [ ]:
# Methods are defined with the same syntax, but with different types
function myfunc(x::Integer, y::Integer)
    x + y
end

In [ ]:
methods(myfunc)

In [ ]:
# Running with floats uses the first version
myfunc(2.0,2.0)

In [ ]:
# Tells which method is used
@which myfunc(2.0, 2.0)

In [ ]:
# and running with integers uses the second one
myfunc(2,2)

In [ ]:
# We did not define this one
myfunc(2,2.0)

The arguments must be precisely of type `Float64` or `Integer`. Other numeric types, such as 32-bit floating-point values, are not automatically converted to 64-bit floating-point, nor are strings parsed as numbers. 

It may often be useful, however, to write more general methods where the declared parameter types are abstract:

In [ ]:
function myfunc(x::Number, y::Number)
    print("Got ", x, " and ", y)
end

In [ ]:
myfunc(2.0, 3)

Julia pick the least general method that matches. So this one still uses the first version:

In [ ]:
myfunc(2.0, 3.0)

## Exercise 4: Add a method

The function below adds any number to any other number.

In [ ]:
add(x::Number, y::Number) = x + y

The word add could have other meanings as well. Write a method of the add function that takes two strings and adds the
second to the end of the first.

So if called with "Hello " and "World!", the function returns "Hello World!".

In [ ]:
add(x::String, y::String) = x * y # alternatively, "$x$y" also works.

add("Hello ", "World!")

Create another method for the add function. This time the first argument is an array and the second argument is a number.

The function returns a new array with the number added to the end of the original array.

In [ ]:
function add(x::Array, y::Number)
    push!(x, y)
end

arr = add([1,2,3],5)

## Exercise 5: List methods

What methods does the add function have?

Check other functions you have used.

In [ ]:
methods(add)

### Manually defined types

Combining multiple dispatch with manually defined types is really powerful:

In [ ]:
"Since MyInt is better than a standard Number, multiply it by 3"
function myfunc(x::Number, y::MyInt)
    return 2*x + 3*y.number
end

"Since MyInt is better than a standard Number, multiply it by 3"
function myfunc(x::MyInt, y::Number)
    result = 3*2*x.number + y
    
    # Return a MyInt type number
    return MyInt(result)
end

In [ ]:
myfunc(5, MyInt(5))

In [ ]:
myfunc(MyInt(5), 5)

### Exercise 6: Rock, paper, scissors

Below we define three types, rock, paper and scissors.

In [ ]:
struct Rock
end

struct Paper
end

struct Scissors
end

Write the function play that prints the winner of the a game between two of these types. For example, `play(rock, paper)` should print "second wins!"

In [ ]:
# Try seeking approaches being more efficient
function play(x::Rock, y::Rock) = "the two players tie."
function play(x::Rock, y::Paper) = "second wins!"
function play(x::Rock, y::Scissors) = "first wins!"

function play(x::Paper, y::Paper)
    println("the two players tie.")
end
function play(x::Paper, y::Scissors)
    println("second wins!")
end
function play(x::Paper, y::Rock)
    println("first wins!")
end
function play(x::Scissors, y::Scissors)
    println("the two players tie.")
end
function play(x::Scissors, y::Rock)
    println("second wins!")
end
function play(x::Scissors, y::Paper)
    println("first wins!")
end

rock = Rock()
paper = Paper()

play(rock, paper)


## Epidemic Simulation

We can extend almost any Julia function with custom behaviour for our custom datatypes.

Case in point, let's print the Plant type in a prettier way.

In [ ]:
# This is how we write a print function for a plant
function Base.show(io::IO, plant::Plant)
    if plant.status == infected
        print(io, "◼")
    else
        print(io, "◻")
    end
end

In [ ]:
# Create an infected and an uninfected plant
plant1 = Plant(uninfected, 0)
plant2 = Plant(infected, 0)

println(plant1, plant2)
interact!(plant1, plant2, 0.5)
println(plant1, plant2)

## Extra Exercise 1: Fruit Bowl

Write functions to keep track of fruit in your fruit bowl and to warn you when it's empty or if no more fruit will fit in. For simplicity, assume a given number of fruit will fit not matter the kind.

To get started, create a new type for a piece of fruit. The fruit can be either an apple, a banana or an orange. (You can add others based on your preference.)

You can use the @enum to represent types of fruit.

In [ ]:
@enum Fruit apple banana orange pear melon

Now create a mutable type called FruitBowl. It contains a list of fruit in the bowl and the maximum capacity of the bowl (integer).

In [ ]:
mutable struct FruitBowl
    list::Array
    capacity::Integer
end

Now create an `add!`-function to add a piece of fruit to the bowl. Check that the bowl does not go over capacity.

In [ ]:
function add!(x::FruitBowl, y::Fruit)
    if length(x.list) ≥ x.capacity
        println("The bowl is full.")
    else
        push!(x.list, y)
    end
    # @assert length(x.list) <= x.capacity "The bowl is full."
end

Test your code by running the two cells below

In [ ]:
# Test it with an empty bowl
bowl = FruitBowl([], 4)
add!(bowl, apple)

In [ ]:
# Keep adding untill the bowl is full
add!(bowl, apple)
add!(bowl, banana)
add!(bowl, orange) 
add!(bowl, orange) # This one does not fit

## Broadcasting
Julia functions and operators do not automatically apply to individual elements of input when it is a collection type (e.g. Array), unlike in Matlab where functions do for matrix input. To enable broadcasting of Julia functions and operators calls for approaches below: 

### Function `map` and `broadcast`

In [ ]:
# compare the difference
@show map(abs, [-1, -2, -3])
@show broadcast(abs, [-1, -2, -3])
@show map(+, 1, [1, 2, 3])
@show broadcast(+, 1, [1, 2, 3])

### Dot notation

In [ ]:
@show 1 .+ [1, 2, 3]
@show abs.([-1, -2, -3])

# Extra:

## Macros
Finally, let's touch the metaprogramming capabilities of Julia. 

Since metaprogramming is a whole other topic (see bonus notebook) we will only cover the very basics of something you might encounter when dealing with Julia code: macros.

For a clear and concise explantion on macros, watch [this video](https://www.youtube.com/watch?v=e6LGMeoQhfs).

Macros provide a method to include generated code in the final body of a program. A macro maps a tuple of arguments to a returned **expression**, and the resulting expression is compiled directly.

This means that macros can change how functions work, hence the *meta* in metaprogramming.

In [ ]:
macro sayhello()
    return :( println("Hello, world!") )
end

## Macro invocation
Macros are invoked with the following general syntax:
```julia
@name expr1 expr2 ...
@name(expr1, expr2, ...)
```

In [ ]:
@sayhello

## Advanced: Tasks (aka Coroutines)
Tasks are a control flow feature that allows computations to be suspended and resumed in a flexible manner. More information can be found from the [documentation](https://docs.julialang.org/en/v1/manual/control-flow/#man-tasks-1)

In [9]:
function producer(c::Channel)
    put!(c, "start")
    put!(c, 1)
    put!(c, 2)
    put!(c, 3)
    put!(c, 3)
    put!(c, "stop")
end;

chnl = Channel(producer);

In [ ]:
take!(chnl) # try executing me repeatedly